In [5]:
# Import Necessary Modules
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import requests
import io
import matplotlib.colors as colors
import matplotlib.cm as cm
import folium
from sklearn.cluster import KMeans
import wikipedia as wp

In [6]:
#Lets encode the wikipedia page using wikipedia module

html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")

#Convert the following into DataFrame

data = pd.read_html(html, header=0)[0]
data.head()

data.drop(data[data["Borough"] == "Not assigned"].index, inplace=True)
data.head()

#If more than one neighborhood exists in one postal code area, then those two rows will combined into one row with the neighborhoods seperated by comma

data = data.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x: ', '.join(x)).to_frame().reset_index()
data.head()

#If a cell has a borough but a Not Assigned neighborhood, then the neighborhood will be same as the borough

for index, row in data.iterrows():
    if row["Neighbourhood"] == 'Not Assigned':
        row["Neighbourhood"] = row["Borough"]

data.shape

(103, 3)

In [13]:
#In order to use Foursquare location data, we need to get the latitude and longitude of corodinates of each neighborhoodurl = "https://cocl.us/Geospatial_data"

s = requests.get(url).content
c = pd.read_csv(io.StringIO(s.decode('utf-8')))

c.columns = ["Postal Code", "Latitude", "Longitude"]
data = pd.merge(c, data, on="Postal Code")

In [14]:
address = "Toronto, Ontario Canada"

geolocator = Nominatim(user_agent="Cana_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical corodinates of Toronto, Ontario Canada are {}, {}".format(latitude, longitude))

The geographical corodinates of Toronto, Ontario Canada are 43.6534817, -79.3839347


In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(data["Latitude"], data["Longitude"], data["Borough"], data["Neighbourhood"]):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup = label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity = 0.7,
            parse_html = False).add_to(map_toronto)

map_toronto

In [19]:
# I will work with the data with only boroughs that contains Toronto.

toronto_data = data[data["Borough"].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(39, 9)


,Postal Code,Latitude,Longitude,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Borough,Neighbourhood
0,M4E,43.676357,-79.293031,43.676357,-79.293031,43.676357,-79.293031,East Toronto,The Beaches
1,M4K,43.679557,-79.352188,43.679557,-79.352188,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
2,M4L,43.668999,-79.315572,43.668999,-79.315572,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
3,M4M,43.659526,-79.340923,43.659526,-79.340923,43.659526,-79.340923,East Toronto,Studio District
4,M4N,43.728020,-79.388790,43.728020,-79.388790,43.728020,-79.388790,Central Toronto,Lawrence Park


In [25]:
#Recreate the map with new markers for Toronto Neighborhoods

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, label in zip(toronto_data["Latitude"], toronto_data["Longitude"], toronto_data["Neighbourhood"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color="blue",
            fill=True,
            fill_color="#3186cc",
            fill_opacity = 0.7,
            parse_html=False).add_to(map_toronto)
    
map_toronto

In [26]:
#Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

CLIENT_ID = '5GVRQXNNTEIRE4VU4TXVFXGOKVJ5FBGNRSXURDJW31I3HVOG' # your Foursquare ID
CLIENT_SECRET = '30XEEC4HVZXOVFOIWSU0SZYESKZUZRVUFOFZWDMW45TTW5EY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5GVRQXNNTEIRE4VU4TXVFXGOKVJ5FBGNRSXURDJW31I3HVOG
CLIENT_SECRET:30XEEC4HVZXOVFOIWSU0SZYESKZUZRVUFOFZWDMW45TTW5EY


In [27]:
# The following function retrieves the venues given the names and coordinates and stores it into dataframe.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
#Retrieve all venues given the address

toronto_neighborhoods = toronto_data
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighbourhood'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [30]:
#Check size of resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(1598, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [31]:
#count of venues were returned of each borough

toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,82,82,82,82,82,82
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,32,32,32,32,32,32


In [32]:
#How many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


# 2. Analyze Each Borough Neighborhood

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## And let's examine the new dataframe size

In [34]:
toronto_onehot.shape

(1598, 240)

In [35]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.016667,0.000000,0.016667,0.000000,0.016667,0.016667
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012195,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012195,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.03125,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
#Check new size:
toronto_grouped.shape

(39, 240)

In [37]:
#Let's put that into a pandas Dataframe
num_top_venues = 5
for neigh in toronto_grouped['Neighbourhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.11
1          Bakery  0.05
2    Cocktail Bar  0.05
3  Farmers Market  0.04
4     Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
               venue  freq
0               Café  0.13
1     Breakfast Spot  0.09
2             Bakery  0.09
3        Coffee Shop  0.09
4  Convenience Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3          Comic Shop  0.06
4    Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.20
1       Airport Lounge  0.13
2     Sculpture Garden  0.07
3  Rental Car Location  0.07
4   Airport Food Court  0.07


----Central Bay Street----
                venue  freq
0       

In [38]:
#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(39, 11)

# Clustering Neighborhoods

In [40]:
#Run k-means to cluster the neighborhood into 6 clusters.

# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[0 7 7 0 0 4 0 0 7 9]
39


In [41]:
toronto_neighborhoods.head()

,Postal Code,Latitude,Longitude,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Borough,Neighbourhood
0,M4E,43.676357,-79.293031,43.676357,-79.293031,43.676357,-79.293031,East Toronto,The Beaches
1,M4K,43.679557,-79.352188,43.679557,-79.352188,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
2,M4L,43.668999,-79.315572,43.668999,-79.315572,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
3,M4M,43.659526,-79.340923,43.659526,-79.340923,43.659526,-79.340923,East Toronto,Studio District
4,M4N,43.728020,-79.388790,43.728020,-79.388790,43.728020,-79.388790,Central Toronto,Lawrence Park


In [42]:
toronto_merged = toronto_neighborhoods

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Latitude,Longitude,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,43.676357,-79.293031,43.676357,-79.293031,43.676357,-79.293031,East Toronto,The Beaches,0,Health Food Store,Trail,Pub,Neighborhood,Ethiopian Restaurant,Escape Room,Electronics Store,Event Space,Diner,Falafel Restaurant
1,M4K,43.679557,-79.352188,43.679557,-79.352188,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",7,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,M4L,43.668999,-79.315572,43.668999,-79.315572,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West",7,Park,Fast Food Restaurant,Movie Theater,Liquor Store,Italian Restaurant,Restaurant,Sushi Restaurant,Fish & Chips Shop,Pub,Ice Cream Shop
3,M4M,43.659526,-79.340923,43.659526,-79.340923,43.659526,-79.340923,East Toronto,Studio District,0,Coffee Shop,Gastropub,Brewery,American Restaurant,Bakery,Café,Middle Eastern Restaurant,Cheese Shop,Coworking Space,Seafood Restaurant
4,M4N,43.728020,-79.388790,43.728020,-79.388790,43.728020,-79.388790,Central Toronto,Lawrence Park,0,Park,Bus Line,Business Service,Swim School,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


# Finally, Lets visualize the clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
